<a href="https://colab.research.google.com/github/ranesh88/Natural-Language-Processing/blob/main/NLP_Asssignment_2(Text_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
stop = stopwords.words('english')
punkt = list(punctuation)
bad_tokens = stop + punkt

In [43]:
df = pd.read_csv("/content/news.csv")

In [44]:
df.head()

,text,subject,fake
0,Donald Trump just couldn t wish all Americans ...,News,1
1,House Intelligence Committee Chairman Devin Nu...,News,1
2,"On Friday, it was revealed that former Milwauk...",News,1
3,"On Christmas day, Donald Trump announced that ...",News,1
4,Pope Francis used his annual Christmas Day mes...,News,1


In [45]:

def clean_text(text):
    
    text = text.lower()
    tokens = word_tokenize(text)
    word_tokens = [t for t in tokens if t.isalpha()]
    lemma = WordNetLemmatizer()
    lemma_tokens = [lemma.lemmatize(w) for w in word_tokens]
    final_tokens = [t for t in word_tokens if t not in lemma_tokens]
    return " ".join(final_tokens)

In [39]:
df["text"] = df["text"].apply(clean_text)

In [7]:
df["text"]

0      americans enemies haters media as friends supp...
1      us was as documents was as has as boys months ...
2      was was has was feelings has copies media atte...
3      games appearances rounds pass https makes has ...
4      days members nations as states borders childre...
                             ...                        
403    us philosophers moralists masochists evangelis...
404    las vegas details vegas questions scenes las v...
405    fascists categories fascists weeks has demonst...
406    protagonists las vegas events las vegas has me...
407    residents states passports flights states us r...
Name: text, Length: 408, dtype: object

In [8]:
X = df["text"]
y = df["fake"]

In [9]:
y.value_counts()

1    204
0    204
Name: fake, dtype: int64

In [10]:
def encode(x):
    index = {"fake":1,"not fake":0}
    return index[x]

In [11]:
y.value_counts()

1    204
0    204
Name: fake, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,test_size=0.3)

In [14]:
cv = CountVectorizer()

In [15]:
X_train_cv = cv.fit_transform(X_train)

In [16]:
X_test_cv = cv.transform(X_test)

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [18]:
dt = DecisionTreeClassifier()

In [19]:
dt.fit(X_train_cv,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [20]:
y_pred = dt.predict(X_test_cv)

In [21]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.24      0.24      0.24        68
           1       0.07      0.07      0.07        55

    accuracy                           0.16       123
   macro avg       0.16      0.15      0.15       123
weighted avg       0.16      0.16      0.16       123



 TF-IDF

In [23]:
tf = TfidfVectorizer()

In [24]:
# X_train_tf = tf.fit_transform(X_train) is enough
X_train_tf = tf.fit_transform(X_train)

In [25]:
X_test_tf = tf.transform(X_test)

In [26]:
dt2 = DecisionTreeClassifier()

In [27]:
dt2.fit(X_train_tf,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [28]:
y_pred = dt2.predict(X_test_tf)

In [29]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.21      0.19      0.20        68
           1       0.11      0.13      0.12        55

    accuracy                           0.16       123
   macro avg       0.16      0.16      0.16       123
weighted avg       0.17      0.16      0.16       123



In [31]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(2, activation="relu", input_shape=(X_train_cv.shape[1],)),
    tf.keras.layers.Dense(3, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
    
])

In [32]:
model.compile(optimizer="adam", loss="binary_crossentropy")

In [33]:
model.fit(X_train_tf.toarray(), y_train, epochs=50, batch_size=50)

Epoch 1/50
6/6 [==============================] - 2s 4ms/step - loss: 0.6931
Epoch 2/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6922
Epoch 3/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6918
Epoch 4/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6913
Epoch 5/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6902
Epoch 6/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6898
Epoch 7/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6905
Epoch 8/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6905
Epoch 9/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6889
Epoch 10/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6884
Epoch 11/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6880
Epoch 12/50
6/6 [==============================] - 0s 3ms/step - loss: 0.6836
Epoch 13/50
6/6 [==============================] - 0s 3ms/step - loss: 0.

In [34]:
y_pred = model.predict(X_test_tf.toarray())

In [36]:
  y_pred = np.where(y_pred >= 0.5,1,0)

In [37]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.12      0.07      0.09        68
           1       0.23      0.35      0.28        55

    accuracy                           0.20       123
   macro avg       0.18      0.21      0.18       123
weighted avg       0.17      0.20      0.17       123

